<div align="center">
    <a href="https://github.com/syubogdanov/hse-howto-python">
        <img src="https://cdn-icons-png.flaticon.com/128/1864/1864510.png" height="128px" width="auto">
    </a>
    <h3>
        <b>
            Продвинутый Python
        </b>
    </h3>
    <i>
        Продвинутое ООП. Часть III
    </i>
</div>

<br>

**Цель занятия.** Продолжение изучения объектно-ориентированного программирования. Занятие включает в себя методы реализации эффективного перебора, а также углубление в наследование классов: доопределение наследуемых методов и разрешение конфликтов наследования.

**Рассуждение.** Когда мы говорим об итерировании по некоторой коллекции, вероятно, первое, что приходит в голову - это цикл `for` вместе с операцией обращения по индексу. В целом, идея неплохая. Для контейнеров, у которых асимптотическая сложность обращения по индексу равна `O(1)`, такой подход будет весьма осознанным и удобным.

**Пример.** Перебор элементов списка при помощи операции обращения к элементу по индексу.

In [1]:
example = [1, 2, 3]

for index in range(len(example)):
    print(example[index])

1
2
3


**Пример.** Перебор элементов стека при помощи операции обращения по индексу.

In [2]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Any, Optional


@dataclass(slots=True)
class Node(object):
    value: Any
    next: Optional[Node]

In [3]:
class Stack(object):
    __slots__: tuple[str, ...] = (
        "_head",
        "_len",
    )

    def __init__(self: Stack) -> None:
        self._len: int = 0
        self._head: Optional[Node] = None

    def append(self: Stack, value: Any) -> None:
        self._head = Node(
            value=value,
            next=self._head,
        )
        self._len += 1

    def pop(self: Stack) -> Any:
        value: Any = self._head.value

        self._head = self._head.next
        self._len -= 1

        return value

    def clear(self: Stack) -> None:
        self._len = 0
        self._head = None

    def __len__(self: Stack) -> int:
        return self._len

    def __getitem__(self: Stack, index: int) -> Any:
        if not (0 <= index < len(self)):
            raise IndexError

        node: Optional[Node] = self._head
        for _ in range(index):
            node = node.next

        return node.value

In [4]:
stack = Stack()

stack.append(3)
stack.append(2)
stack.append(1)

for index in range(len(stack)):
    print(stack[index])

1
2
3


**Замечание.** Если корректно определить магические методы `__len__` и `__getitem__`, тогда интерпретатор самостоятельно добавит возможность использовать Ваш класс в цикле `for` для перебора элементов.

In [5]:
stack = Stack()

stack.append(3)
stack.append(2)
stack.append(1)

for value in stack:
    print(value)

1
2
3


**Замечание.** Сейчас операция обращения по индексу имеет асимптотическую сложность `O(N)`. Это значит, что в текущей формации перебор элементов стека осуществляется за асимптотическую сложность <code>O(N<sup>2</sup>)</code>. В то же время реализовать перебор возможно и за линейную сложность.

**Рассуждение.** Давайте для итерирования по структуре реализуем метод `iterate`, который будет возвращать список, состоящий из элементов стека. С одной стороны, получаем, что асимптотическая сложность перебора сократилась до `O(N)`. С другой стороны, получаем явную зависимость от памяти. Например, если стек будет состоять из очень большого числа элементов, то может получиться так, что данные не влезут в оперативную память и программа завершит свое исполнение из-за нехватки ресурсов.

**Определение.** Итератор - это объект, представляющий поток данных. Итератор отвечает следующим требованиям:

- Определен магический метод `__iter__`, который возвращает итератор;
- Определен магический метод `__next__`, который последовательно возвращает элементы потока;
- Если поток данных окончен, тогда использование `__next__` порождает исключение `StopIteration`.

**Пояснение.** В некотором смысле, итератор - это указатель на первый элемент объекта, к которому был применен метод `__iter__`. Используя метод `__next__`, Вы, во-первых, получаете значение элемента, на который указывал итератор, а во-вторых, сдвигаете указатель к следующему элементу.

**Пояснение.** Такая сущность, как итератор, помогает разработчику реализовать интерфейс эффективного как по памяти, так и по скорости перебора. Концепция итераторов предполагает, что Вы, во-первых, будете возвращать элементы коллекции по одному. За счет этого достигается оптимальность по памяти. А во-вторых, предполагается, что переход от текущего элемента к следующему осуществляется за минимально возможное время. Как итог, получаем максимальную эффективность при минимальных затратах.

**Пример.** Перебор элементов списка при помощи неявного применения итераторов.

In [6]:
example = [1, 2, 3]

for value in example:
    print(value)

1
2
3


**Пояснение.** Когда Вы используете итерирование по структуре при помощи цикла `for`, интерпретатор неявно вызывает магический метод `__iter__`, который возвращает итератор по коллекции. Далее, он перебирает все элементы итератора до тех пор, пока не получит исключение `StopIteration`. Если же метода `__iter__` нет, то интерпетатор пытается перебрать элементы при помощи методов `__len__` и `__getitem__`.

**Пример.** Перебор элементов списка при помощи явного применения итераторов.

In [7]:
example = [1, 2, 3]

iterator = iter(example)     # Получить итератор

print("->", next(iterator))  # 1-ый элемент списка
print("->", next(iterator))  # 2-ой элемент списка
print("->", next(iterator))  # 3-ий элемент списка

-> 1
-> 2
-> 3


**Упражнение.** Попробуйте вызвать `next(iterator)` еще один раз.

**Упражнение.** Вернитесь к примеру с перебором стека. Попробуйте вызвать магический метод `__iter__`. Получите ли Вы итератор?

**Пример.** Перебор элементов множества при помощи явного применения итераторов.

In [8]:
example = {1, 2, 3, 4, 5, 6}

iterator = iter(example)  # Получить итератор

print(next(iterator))  # 1-ый элемент множества
print(next(iterator))  # 2-ой элемент множества
print(next(iterator))  # 3-ий элемент множества

print(next(iterator))  # 4-ый элемент множества
print(next(iterator))  # 5-ый элемент множества
print(next(iterator))  # 6-ой элемент множества

1
2
3
4
5
6


**Пример.** Проверка принадлежности числа списку на основе итераторов.

In [9]:
example = {1, 2, 3, 4, 5, 6}

iterator = iter(example)

print("4 in iterator:", 4 in iterator)
print("4 in iterator:", 4 in iterator)

4 in iterator: True
4 in iterator: False


**Пояснение.** Сейчас мы наблюдаем очень странную, на первый взгляд, картину - в первом случае получили `True`, а во втором - уже `False`, хотя код никак не был изменен. Чтобы понять, почему так происходит, давайте вспомним, что делают итераторы.

Когда мы применяем магический метод `__next__` (используется выше в неявном виде), итератор возвращает сначала текущий элемент, а после этого сдвигает "указатель" к следующему элементу. Еще раз. К следующему элементу. Есть ли после первой четверки равные ей элементы? Нет!

Поясним чуть подробнее. Когда мы применяем поиск по итератору впервые, магический метод `__next__`, дойдя до четверки, остановится, потому что нашел подходящее значение. В тот момент итератор уже будет указывать на пятерку (как на следующий элемент). Повторное использование итератора приведет к тому, что все элементы начнут перебираться именно с пятерки, а не с самого начала, потому что итераторы идут всегда к следующему элементу.

**Примечание.** Будьте предельно внимательны при работе с итераторами. Их опустошение нередко становится головной болью неопытных разработчиков.

**Пример.** Пример опустошения итератора.

In [10]:
example = {1, 2, 3, 4, 5, 6}

iterator = iter(example)

print(">", next(iterator), "<")
print(">", next(iterator), "<")

for value in iterator:
    print(value)

> 1 <
> 2 <
3
4
5
6


**Замечание.** Внимательный читатель вспомнит, что `for` должен вызывать магический метод `__iter__`. Все так. Дело в том, что итераторы устроены таким образом, что их метод `__iter__` всегда возвращает сам объект, а не его копию-итератор.

**Пример.** Реализация перебора при помощи явного применения итераторов.

In [11]:
example = [1, 2, 3]

iterator = iter(example)

while True:
    try:
        print(next(iterator))
    except StopIteration:
        break

1
2
3


**Пояснение.** С идейной точки зрения, именно таким способом реализован перебор коллекций, для которых определен магический метод `__iter__`.

**Пример.** Реализуйте итератор, который перебирает `n` случайных чисел в диапазоне от `a` до `b`.

In [12]:
import random

from collections.abc import Iterator
from typing import Self


class RandomIterator(Iterator):
    __slots__: list[str, ...] = [
        "__lh_bound",  # Левая граница для случайных значений
        "__rh_bound",  # Правая граница для случайных значений
        "__left",      # Сколько значений нужно еще вернуть
    ]

    def __init__(self: Self, n: int, a: int, b: int) -> None:
        self.__lh_bound: int = a
        self.__rh_bound: int = b
        self.__left: int = n

    def __iter__(self: Self) -> Iterator[int]:
        return self

    def __next__(self: Self) -> int:
        if self.__left <= 0:
            raise StopIteration

        self.__left -= 1
        return random.randint(self.__lh_bound, self.__rh_bound)

In [13]:
for value in RandomIterator(n=5, a=-10, b=10):
    print(value)

3
10
-6
-5
3


**Пояснение.** Для итератора, как уже было сказано, достаточно определить всего лишь два магических метода - `__iter__` и `__next__`. В методе `__iter__`, в котором мы получаем итератор, достаточно вернуть себя - об этом уже было сказано выше. Когда же мы говорим о магическом методе `__next__`, то нужно выполнить всего два условия: передавать данные, пока возможно, иначе - бросать исключение `StopIteration`.

**Замечание.** Настоятельно рекомендуется реализовывать итераторы в виде отдельных сущностей.

**Пример.** Реализация итератора для стека.

In [16]:
class StackIterator(Iterator):
    __slots__: tuple[str] = ("__node")

    def __init__(self: Self, stack: Stack) -> None:
        self.__node: Optional[Node] = stack._head

    def __iter__(self: Self) -> Iterator[Any]:
        return self

    def __next__(self: Self) -> Any:
        if self.__node is None:
            raise StopIteration

        value: Any = self.__node.value
        self.__node = self.__node.next

        return value


class Stack(object):
    __slots__: tuple[str, ...] = (
        "_head",
        "_len",
    )

    def __init__(self: Stack) -> None:
        self._len: int = 0
        self._head: Optional[Node] = None

    def __iter__(self: Stack) -> StackIterator:
        return StackIterator(self)

    def append(self: Stack, value: Any) -> None:
        self._head = Node(
            value=value,
            next=self._head,
        )
        self._len += 1

    def pop(self: Stack) -> Any:
        value: Any = self._head.value

        self._head = self._head.next
        self._len -= 1

        return value

    def clear(self: Stack) -> None:
        self._len = 0
        self._head = None

    def __len__(self: Stack) -> int:
        return self._len

    def __getitem__(self: Stack, index: int) -> Any:
        if not (0 <= index < len(self)):
            raise IndexError

        node: Optional[Node] = self._head
        for _ in range(index):
            node = node.next

        return node.value

In [17]:
stack = Stack()

stack.append(3)
stack.append(2)
stack.append(1)

for value in stack:
    print(value)

1
2
3


**Пояснение.** В примере выше, во-первых, был реализован итератор `StackIterator`, который эффективно перебирает элементы стека, начиная с его вершины. Самое важное на этом этапе - не забыть определить метод `__iter__` в основном классе. В остальном все достаточно просто - храним ссылку на требуемый узел, а при необходимости - передвигаемся к следующему элементу.

In [18]:
stack = Stack()

stack.append(3)
stack.append(2)
stack.append(1)

iterator = iter(stack)
print(type(iterator))

print(next(iterator))
print(next(iterator))
print(next(iterator))

<class '__main__.StackIterator'>
1
2
3


**Пример.** Реализуйте итератор, который перебирает факториалы чисел от `0` до `n`.

In [19]:
class FactorialIterator(Iterator):
    __slots__: tuple[str, ...] = (
        "__factorial",
        "__number",
        "__left",
    )

    def __init__(self: Self, n: int) -> None:
        self.__factorial: int = 1  # Текущий факториал
        self.__number: int = 1     # Следующее число
        self.__left: int = n       # Сколько осталось вывести

    def __iter__(self: Self) -> FactorialIterator:
        return self

    def __next__(self: Self) -> int:
        if self.__left <= 0:
            raise StopIteration

        value: int = self.__factorial
        self.__factorial *= self.__number
        self.__number += 1
        self.__left -= 1

        return value

In [20]:
for value in FactorialIterator(10):
    print(value)

1
1
2
6
24
120
720
5040
40320
362880


**Рассуждение.** Как можно заметить, итераторы - достаточно громоздкие объекты, с точки зрения реализации. Чтобы написать итератор по факториалам, пришлось задействовать порядка 25 строк кода. Для удобства разработчиков придумали генераторы.

**Определение.** Генератор - это особый вид функций, реализующий поведение итератора. Генератор обладает следующими отличительными чертами:
- Для возвращения значений вместо `return` используется `yield`;
- В функциях-генераторах `return value` аналогичен `raise StopIteration(value)` в итераторах;
- Вызов `yield` возвращает значение из генератора и запоминает состояние (то есть переменные), в котором он был перед отправкой значения. Последующее исполнение начинается с того места, где закончилось предыдущее.

**Пояснение.** Говоря максимально просто, функция-генератор - это такой итератор, который самостоятельно запоминает контекст исполнения. В частности, более не требуется заполнять какие-то внутрение поля класса-итератора - все будет исполнено автоматически.

**Замечание.** В отличие от функций, генераторы имеют более сложную аннотацию. В рамках курса используйте аннотацию `Generator[YieldType, None, ReturnType]`. Значение, которое находится посередине, отвечает за отправляемое в генератор значение. Это связано с объектом, имеющим название "корутина". В рамках текущего курса мы его изучать не будем.

**Пример.** Реализуйте генератор, который перебирает `n` случайных чисел в диапазоне от `a` до `b`.

In [21]:
from collections.abc import Generator

def iterate_random(n: int, a: int, b: int) -> Generator[int, None, None]:
    for _ in range(n):
        yield random.randint(a, b)

In [23]:
for value in iterate_random(n=5, a=-10, b=10):
    print(value)

4
-10
9
-7
-3


**Пояснение.** Возможно, Вам станет немного понятнее, если мы рассмотрим генератор как итератор. Вновь напомним, что это не совсем обычные функции.

In [25]:
iterator = iter(iterate_random(n=5, a=-10, b=10))

print(next(iterator))
print(next(iterator))
print(next(iterator))
print(next(iterator))
print(next(iterator))

-7
5
-7
-10
6


**Определение.** Итерируемый объект - это объект, у которого определен магический метод `__iter__`.

**Упражнение.** Сравните определения *итератор* и *итерируемый объект*. Что между ними общего? Чем они отличаются? Является ли итератор итерируемым объектом? А наоборот? Почему?

**Пример.** Реализовать генератор для перебора четных целых чисел в произвольном итерируемом объекте.

In [26]:
from collections.abc import Iterable

def iterate_even(iterable: Iterable[Any]) -> Generator[int, None, None]:
    for value in iterable:
        if not isinstance(value, int):
            continue

        if value % 2 != 0:
            continue

        yield value

In [27]:
example = (1, 2, hex(3), "4", 5, bin(5), [6], 7, 8)

for value in iterate_even(example):
    print(value)

2
8


**Пример.** Реализовать генератор по лексикографически не убывающему префиксу строки.

In [28]:
def mygenerator(string: str) -> Generator[str, None, None]:
    if not string:
        return

    last: str = string[0]
    for letter in string:

        if letter < last:
            return

        last = letter
        yield last

In [29]:
for letter in mygenerator("aaabcda_stop"):
    print(letter)

a
a
a
b
c
d


**Пояснение.** Использование `return` в функциях-генераторах, как уже упоминалось, равносильно тому, что Вы выбросили исключение `StopIteration` в итераторе. Например, когда на вход подается пустая строка, можно сразу выходить из генератора, так как ничего более не будет совершено. Или, например, когда нарушен лекискографический порядок из условия - аналогично.

**Пример.** Реализовать генератор, который перебирает только `k`-ые элементы произвольного итерируемого объекта.

In [44]:
def mygenerator(k: int, iterable: Iterable[Any]) -> Generator[int, None, None]:
    index: int = -1
    for value in iterable:
        index += 1

        if index % k != 0:
            continue

        yield value

In [46]:
example = ["1", 2, (3), bin(4), [5], {6}, hex(7)]
kth = 3

for value in mygenerator(kth, example):
    print(value)

1
0b100
0x7


**Пример.** Опустошение генератора.

In [47]:
def mygenerator(n: int) -> Generator[int, None, None]:
    for _ in range(n):
        yield 1

In [48]:
example = [1, 1, 1, 1, 1]
ones = mygenerator(n=5)

print("Первое значение генератора:", next(ones))
print("Второе значение генератора:", next(ones))

print("Список на основе генератора:", list(ones))

Первое значение генератора: 1
Второе значение генератора: 1
Список на основе генератора: [1, 1, 1]


**Пример.** Опустошение генератора.

In [49]:
def mygenerator(n: int) -> Generator[int, None, None]:
    for value in range(1, n + 1):
        yield value

In [50]:
iterator = iter(mygenerator(10))

print("5 in iterator:", 5 in iterator)
print("5 in iterator:", 5 in iterator)

5 in iterator: True
5 in iterator: False


**Рассуждение.** Давайте вернемся к наследованию классов. Порой может возникнуть задача, когда в дочернем классе нужно расширить родительский метод, а не переопределить его. Приведем пример. Положим, что имеется реализация узла для односвязного списка. Тогда, чтобы расширить определение до двусвязности, достаточно лишь доопределить инициализатор узла.

**Пример.** Реализация узла односвязного списка.

In [8]:
class LinkedListNode(object):
    __slots__: tuple[str, ...] = (
        "__value",
        "__next",
    )

    def __init__(
        self: LinkedListNode,
        value: Any,
        next: Optional[LinkedListNode] = None,
    ) -> None:
        self.__value: Any = value
        self.__next: Optional[LinkedListNode] = next

    @property
    def value(self: LinkedListNode) -> Any:
        return self.__value

    @value.setter
    def value(self: LinkedListNode, value: Any) -> None:
        self.__value = value

    @property
    def next(self: LinkedListNode) -> Optional[LinkedListNode]:
        return self.__next

**Пример.** Доопределение узла односвязного списка до двусвязности.

In [9]:
class DoublyLinkedListNode(LinkedListNode):
    __slots__: tuple[str] = ("__previous")

    def __init__(
        self: DoublyLinkedListNode,
        value: Any,
        next: Optional[LinkedListNode] = None,
        previous: Optional[LinkedListNode] = None,
    ) -> None:
        super().__init__(value=value, next=next)
        self.__previous: Optional[LinkedListNode] = previous

    @property
    def previous(self: DoublyLinkedListNode) -> Optional[DoublyLinkedListNode]:
        return self.__previous

In [10]:
node = DoublyLinkedListNode("Random Value")
node.value

'Random Value'

**Пояснение.** При помощи функции `super()` можно обратиться к экземпляру родительского класса. В примере выше такой вызов позволил сначала инициализировать атрибуты, которые были определены в родителе, а после - доопределить требуемые.

**Пример.** Наследование квадрата от прямоугольника.

In [3]:
class Rectangle(object):
    __slots__: tuple[str, ...] = (
        "__width",
        "__height",
    )

    def __init__(self: Rectangle, width: int, height: int) -> None:
        self.__width: int = width
        self.__height: int = height

    @property
    def width(self: Rectangle) -> int:
        return self.__width
    
    @property
    def height(self: Rectangle) -> int:
        return self.__height

In [4]:
class Square(Rectangle):
    __slots__: tuple[str, ...] = ()

    def __init__(self: Square, side: int) -> None:
        super().__init__(width=side, height=side)

In [6]:
square = Square(side=10)

print("Height:", square.height)
print("Width:", square.width)

Height: 10
Width: 10


**Примечание.** В основном `super()` используют только для того, чтобы дополнить функционал родительского метода.

**Замечание.** Проверить наличие метода или атрибута у объекта можно при помощи функции `hasattr(obj, "method/attr")`. В частности, при помощи `hasattr(super(), "method/attr")` можно узнать наличие метода или атрибута в родительском классе.

**Упражнение.** Попробуйте обратиться к несуществующему атрибуту или методу при помощи `super()`. Какое исключение будет вызвано?

**Рассуждение.** Теперь представим, что у класса несколько родителей. Как в таком случае будет вести себя `super()`? Для этого определили порядок разрешения конфликтов наследования (MRO). Это специальный объект, который определяет порядок классов, в котором дочерний будет искать методы или атрибуты в родительском. Список доступен при обращении к атрибуту `__mro__` у класса (не экземпляра).

In [11]:
class Animal(object):
    pass

class Mammal(Animal):
    pass

class Monkey(Mammal):
    pass

In [17]:
Monkey.__mro__

(__main__.Monkey, __main__.Mammal, __main__.Animal, object)

**Пояснение.** Благодаря MRO можно увидеть, в каком порядке `super()` будет искать методы или атрибуты родительского класса:

1. Если метод определен в `Mammal`, то `super()` использует его;
2. Если метод не определен в `Mammal`, но определен в `Animal`, то `super()` обратится к `Animal`;
3. Если метод не определен ни в `Mammal`, ни в `Animal`, то поиск перейдет к `object`.

Если ни один из трех случаев не подошел - получите исключение.

**Упражнение.** Как определен MRO при изумрудном наследовании?

**Пример.** Вызов метода или атрибута из конкретного родителя.

In [22]:
class Animal(object):
    def whoami(self: Animal) -> None:
        print("Animal")


class Mammal(Animal):
    def whoami(self: Mammal) -> None:
        print("Mammal")


class Monkey(Mammal):
    def whoami(self: Monkey) -> None:
        Animal.whoami(self)

In [23]:
monkey = Monkey()
monkey.whoami()

Animal


**Пояснение.** Достаточно напрямую обратиться к методу нужного класса, предоставив текущий экземпляр в качестве аргумента (вместо `self`).

**Спойлер.** На семинаре:

- Разберете дополнительные примеры работы с итераторами;
- Разберете дополнительные примеры работы с генераторами;
- Вспомните генераторные выражения.